In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 100)

import warnings
warnings.filterwarnings("ignore")

In [2]:
# 용량이 클 경우 chunk 사용 방법

filename = '/fastcampus-data/log-data/csv_seg_all.csv'
chunksize = 10 ** 6

df_list = []
with pd.read_csv(filename, chunksize=chunksize) as reader:
    for chunk in tqdm(reader):
        df_list.append(chunk)
    df = pd.concat(df_list)

df = df.reset_index( drop=True ) 

4it [00:33,  8.38s/it]


In [3]:
df = df[['userid', 'traits_customerid', 'name', 'context_page_path', 'context_page_search', 'context_page_search_keyword', 'context_page_url', 'ymd', 'properties_url', 'type']]

# preprocessing

In [4]:
# split으로 마지막 path만 남기기
df['context_page_path'] = df['context_page_path'].apply(lambda x: x.split('/')[-1])

In [ ]:
# 모든 공백 제거하기
df['context_page_path'] = df['context_page_path'].apply(lambda x: x.replace(" " , ""))

In [ ]:
# 22700%C2%A0 같은 값 떄문에 특수문자 제거
import re
df['context_page_path'] = df['context_page_path'].apply(lambda x: re.sub("[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]", " ", x))

In [ ]:
# path가 모두 숫자이면 int type으로 바꿔주기
df['context_page_path'] = df['context_page_path'].apply(lambda x: int(x) if x[:].isdigit() == True else str(x)  )

In [ ]:
k = []
for i in df['context_page_path']:
    if type(i) == str:
        k.append(i)
print(len(list(set(k)))) # 공백 없애기 전 62개
print(list(set(k)))

# EDA

In [ ]:
print(df.userid.nunique())
# traits_customerid가 있는 userid nunique
df[df['traits_customerid'].notna()].userid.nunique()  # traits_customerid 모두 유니크함.

In [ ]:
df[df['traits_customerid'].notna()].traits_customerid.nunique()

# groupby by userid

In [ ]:
path_group = df.groupby('userid')['context_page_path'].unique().reset_index()

In [ ]:
path_group_set = path_group.context_page_path.apply(lambda x: list(set(x)))

In [ ]:
path_group['context_page_path'].value_counts()

In [ ]:
path_group_set